## Definindo o Problema

In [1]:
#!pip install lightgbm;
#!pip install pymoo;
#!pip install -U pyrecorder;

In [1]:
import pickle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
#from yellowbrick.classifier import ConfusionMatrix
import numpy as np
from tqdm.notebook import tqdm as tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, KFold
#from google.colab import drive
import pickle
from pymoo.algorithms.soo.nonconvex.pso import PSO, PSOAnimation
#from pymoo.factory import Rastrigin
from pymoo.optimize import minimize
import matplotlib.pyplot as plt
from pymoo.factory import get_termination
#from pymoo.util.display import Display
from pymoo.core.callback import Callback

In [2]:
import lightgbm as lgb

In [4]:
#drive.mount('/content/gdrive')

In [3]:
#with open('/content/gdrive/MyDrive/datasets/credit.pkl', 'rb') as f:
#    x_fruit_train, x_fruit_test, y_fruit_train, y_fruit_test = pickle.load(f)
    
#with open('./fruit.pkl', 'rb') as f:
#    x_fruit, y_fruit = pickle.load(f)
    
with open('./fruit.pkl', 'rb') as f:
    x_fruit, y_fruit = pickle.load(f)

Definindo as restrições 

In [6]:
OBJECTIVE = ['Accuracy']
DIMENSIONS = 25 #num_leaves, min_child_samples, n_estimators, learning_rate, subsample_for_bin, min_split_gain, min_child_weight, subsample, reg_alpha, reg_lambda
ITERATIONS = 10
POPULATION = 15
num_leaves_min = 10
num_leaves_max = 200
min_child_samples_min = 10
min_child_samples_max = 50
n_estimators_min = 20
n_estimators_max = 1000

learning_rate_min = 0.001
learning_rate_max = 0.5
subsample_for_bin_min = 50000
subsample_for_bin_max = 500000
min_split_gain_min = 0.01
min_split_gain_max = 0.5
min_child_weight_min = 0.001
min_child_weight_max = 0.15
subsample_min = 1.0
subsample_max = 2.0
reg_alpha_min =0.01
reg_alpha_max = 100.0 
reg_lambda_min = 0.01
reg_lambda_max = 100.0

BOUNDS = []
SEED = 1
INDIVIDUALS = []

https://lightgbm.readthedocs.io/en/latest/Parameters.html#force_col_wise

parameters

In [7]:
max_depth_min = 1
bagging_fraction_min = 0.0
pos_bagging_fraction_min = 0.0
neg_bagging_fraction_min = 0.0
bagging_freq_min = 0
feature_fraction_min = 0.0
feature_fraction_bynode_min = 0.0
extra_trees_min = 0 # boolean
first_metric_only_min = 0 # boolean
max_delta_step_min = -10.0
linear_lambda_min = 0.0
min_data_per_group_min = 1
max_cat_threshold_min = 1
cat_l2_min = 0.0
cat_smooth_min = 0.0
max_cat_to_onehot_min = 1
top_k_min = 1

max_depth_max = 50 #int
bagging_fraction_max = 1.0
pos_bagging_fraction_max = 1.0
neg_bagging_fraction_max = 1.0
bagging_freq_max = 100 #int
feature_fraction_max = 1.0
feature_fraction_bynode_max = 1.0
extra_trees_max = 1 # boolean
first_metric_only_max = 1 # boolean
max_delta_step_max = 10.0
linear_lambda_max = 10.0
min_data_per_group_max = 200 #int
max_cat_threshold_max = 100 #int
cat_l2_max = 20.0
cat_smooth_max = 20.0
max_cat_to_onehot_max = 20 #int
top_k_max = 40 #int

Definindo o problema

In [8]:
import numpy as np
from pymoo.core.problem import ElementwiseProblem

class OptimizeWithAccuracy(ElementwiseProblem):

    def __init__(self):
        super().__init__(n_var= DIMENSIONS,
                         n_obj=1,
                         n_constr=25,
                         xl=np.array([num_leaves_min, min_child_samples_min, n_estimators_min, learning_rate_min, subsample_for_bin_min,
                                      min_split_gain_min, min_child_weight_min, reg_alpha_min, reg_lambda_min, max_depth_min,
                                        bagging_fraction_min,
                                        pos_bagging_fraction_min,
                                        neg_bagging_fraction_min,
                                        bagging_freq_min,
                                        feature_fraction_min,
                                        feature_fraction_bynode_min,
                                        extra_trees_min,
                                        max_delta_step_min,
                                        linear_lambda_min,
                                        min_data_per_group_min,
                                        max_cat_threshold_min,
                                        cat_l2_min,
                                        cat_smooth_min,
                                        max_cat_to_onehot_min,
                                        top_k_min]),
                         xu=np.array([num_leaves_max, min_child_samples_max, n_estimators_max, learning_rate_max, subsample_for_bin_max,
                                      min_split_gain_max, min_child_weight_max, reg_alpha_max, reg_lambda_max,
                                      max_depth_max,
                                        bagging_fraction_max,
                                        pos_bagging_fraction_max,
                                        neg_bagging_fraction_max,
                                        bagging_freq_max,
                                        feature_fraction_max,
                                        feature_fraction_bynode_max,
                                        extra_trees_max,
                                        max_delta_step_max,
                                        linear_lambda_max,
                                        min_data_per_group_max,
                                        max_cat_threshold_max,
                                        cat_l2_max,
                                        cat_smooth_max,
                                        max_cat_to_onehot_max,
                                        top_k_max])
                        )

    def _evaluate(self, x, out, *args, **kwargs):
        #num_leaves, min_child_samples, n_estimators, learning_rate, subsample_for_bin, min_split_gain, min_child_weight, reg_alpha, reg_lambda
        
        model_lgb = lgb.LGBMClassifier(num_leaves        = int(np.round(x[0])), 
                                       min_child_samples = int(np.round(x[1])), 
                                       n_estimators      = int(np.round(x[2])),
                                       learning_rate     = x[3] ,
                                       subsample_for_bin = int(np.round(x[4])),
                                       min_split_gain    = x[5],
                                       min_child_weight  = x[6],
                                       reg_alpha         = x[7],
                                       reg_lambda        = x[8],
                                       max_depth = int(np.round(x[9])),
                                       bagging_fraction = x[10],
                                       pos_bagging_fraction = x[11],
                                       neg_bagging_fraction = x[12],
                                       bagging_freq = int(np.round(x[13])), 
                                       feature_fraction = x[14],
                                       feature_fraction_bynode = x[15],
                                       extra_trees = bool(np.round(x[16])), #boolean
                                       max_delta_step = x[17],
                                       linear_lambda = x[18],
                                       min_data_per_group = int(np.round(x[19])), 
                                       max_cat_threshold = int(np.round(x[20])), 
                                       cat_l2 = x[21],
                                       cat_smooth = x[22],
                                       max_cat_to_onehot = int(np.round(x[23])),
                                       top_k = int(np.round(x[24])),
                                       n_jobs = -1,
                                       )
        
        kfold = KFold(n_splits = 3, shuffle = True)
        
        scores = cross_val_score(model_lgb, x_fruit, y_fruit, cv = kfold, n_jobs=-1)  
        
        result = scores.mean()
        out['F'] = -1 * result

problemAccuracy = OptimizeWithAccuracy()

In [9]:
class OptimizeWithF1(ElementwiseProblem):

    def __init__(self):
        super().__init__(n_var= DIMENSIONS,
                         n_obj=1,
                         n_constr=25,
                         xl=np.array([num_leaves_min, min_child_samples_min, n_estimators_min, learning_rate_min, subsample_for_bin_min,
                                      min_split_gain_min, min_child_weight_min, reg_alpha_min, reg_lambda_min, max_depth_min,
                                        bagging_fraction_min,
                                        pos_bagging_fraction_min,
                                        neg_bagging_fraction_min,
                                        bagging_freq_min,
                                        feature_fraction_min,
                                        feature_fraction_bynode_min,
                                        extra_trees_min,
                                        max_delta_step_min,
                                        linear_lambda_min,
                                        min_data_per_group_min,
                                        max_cat_threshold_min,
                                        cat_l2_min,
                                        cat_smooth_min,
                                        max_cat_to_onehot_min,
                                        top_k_min]),
                         xu=np.array([num_leaves_max, min_child_samples_max, n_estimators_max, learning_rate_max, subsample_for_bin_max,
                                      min_split_gain_max, min_child_weight_max, reg_alpha_max, reg_lambda_max,
                                      max_depth_max,
                                        bagging_fraction_max,
                                        pos_bagging_fraction_max,
                                        neg_bagging_fraction_max,
                                        bagging_freq_max,
                                        feature_fraction_max,
                                        feature_fraction_bynode_max,
                                        extra_trees_max,
                                        max_delta_step_max,
                                        linear_lambda_max,
                                        min_data_per_group_max,
                                        max_cat_threshold_max,
                                        cat_l2_max,
                                        cat_smooth_max,
                                        max_cat_to_onehot_max,
                                        top_k_max])
                        )

    def _evaluate(self, x, out, *args, **kwargs):
        #num_leaves, min_child_samples, n_estimators, learning_rate, subsample_for_bin, min_split_gain, min_child_weight, reg_alpha, reg_lambda
        
        model_lgb = lgb.LGBMClassifier(num_leaves        = int(np.round(x[0])), 
                                       min_child_samples = int(np.round(x[1])), 
                                       n_estimators      = int(np.round(x[2])),
                                       learning_rate     = x[3] ,
                                       subsample_for_bin = int(np.round(x[4])),
                                       min_split_gain    = x[5],
                                       min_child_weight  = x[6],
                                       reg_alpha         = x[7],
                                       reg_lambda        = x[8],
                                       max_depth = int(np.round(x[9])),
                                       bagging_fraction = x[10],
                                       pos_bagging_fraction = x[11],
                                       neg_bagging_fraction = x[12],
                                       bagging_freq = int(np.round(x[13])), 
                                       feature_fraction = x[14],
                                       feature_fraction_bynode = x[15],
                                       extra_trees = bool(np.round(x[16])), #boolean
                                       max_delta_step = x[17],
                                       linear_lambda = x[18],
                                       min_data_per_group = int(np.round(x[19])), 
                                       max_cat_threshold = int(np.round(x[20])), 
                                       cat_l2 = x[21],
                                       cat_smooth = x[22],
                                       max_cat_to_onehot = int(np.round(x[23])),
                                       top_k = int(np.round(x[24])),
                                       n_jobs = -1,
                                       )

        kfold = KFold(n_splits = 3, shuffle = True)
        
        scores = cross_val_score(model_lgb, x_fruit, y_fruit, cv = kfold, scoring='f1_weighted', n_jobs=-1)  
        
        result = scores.mean()
        out['F'] = -1 * result

problemF1 = OptimizeWithF1()

In [4]:
lgb_mean = []
for i in range(30):
    model_lgb = lgb.LGBMClassifier(
                                    n_jobs = -1,
                                    )

    kfold = KFold(n_splits = 3, shuffle = True)

    scores = cross_val_score(model_lgb, x_fruit, y_fruit, cv = kfold, scoring='f1_weighted', n_jobs=-1)  

    result = scores.mean()
    
    lgb_mean.append(result)
    
lgb_mean# Convert the Python array to a numpy array
numpy_array = np.array(lgb_mean)

# Calculate the average value using numpy.mean()
average_value = np.mean(numpy_array)

print("'", average_value, "'")

' 0.8936764606153073 '


In [10]:
class OptimizeWithAUC(ElementwiseProblem):

    def __init__(self):
        super().__init__(n_var= DIMENSIONS,
                         n_obj=1,
                         n_constr=25,
                         xl=np.array([num_leaves_min, min_child_samples_min, n_estimators_min, learning_rate_min, subsample_for_bin_min,
                                      min_split_gain_min, min_child_weight_min, reg_alpha_min, reg_lambda_min, max_depth_min,
                                        bagging_fraction_min,
                                        pos_bagging_fraction_min,
                                        neg_bagging_fraction_min,
                                        bagging_freq_min,
                                        feature_fraction_min,
                                        feature_fraction_bynode_min,
                                        extra_trees_min,
                                        max_delta_step_min,
                                        linear_lambda_min,
                                        min_data_per_group_min,
                                        max_cat_threshold_min,
                                        cat_l2_min,
                                        cat_smooth_min,
                                        max_cat_to_onehot_min,
                                        top_k_min]),
                         xu=np.array([num_leaves_max, min_child_samples_max, n_estimators_max, learning_rate_max, subsample_for_bin_max,
                                      min_split_gain_max, min_child_weight_max, reg_alpha_max, reg_lambda_max,
                                      max_depth_max,
                                        bagging_fraction_max,
                                        pos_bagging_fraction_max,
                                        neg_bagging_fraction_max,
                                        bagging_freq_max,
                                        feature_fraction_max,
                                        feature_fraction_bynode_max,
                                        extra_trees_max,
                                        max_delta_step_max,
                                        linear_lambda_max,
                                        min_data_per_group_max,
                                        max_cat_threshold_max,
                                        cat_l2_max,
                                        cat_smooth_max,
                                        max_cat_to_onehot_max,
                                        top_k_max])
                        )

    def _evaluate(self, x, out, *args, **kwargs):
        #num_leaves, min_child_samples, n_estimators, learning_rate, subsample_for_bin, min_split_gain, min_child_weight, reg_alpha, reg_lambda
        
        model_lgb = lgb.LGBMClassifier(num_leaves        = int(np.round(x[0])), 
                                       min_child_samples = int(np.round(x[1])), 
                                       n_estimators      = int(np.round(x[2])),
                                       learning_rate     = x[3] ,
                                       subsample_for_bin = int(np.round(x[4])),
                                       min_split_gain    = x[5],
                                       min_child_weight  = x[6],
                                       reg_alpha         = x[7],
                                       reg_lambda        = x[8],
                                       max_depth = int(np.round(x[9])),
                                       bagging_fraction = x[10],
                                       pos_bagging_fraction = x[11],
                                       neg_bagging_fraction = x[12],
                                       bagging_freq = int(np.round(x[13])), 
                                       feature_fraction = x[14],
                                       feature_fraction_bynode = x[15],
                                       extra_trees = bool(np.round(x[16])), #boolean
                                       max_delta_step = x[17],
                                       linear_lambda = x[18],
                                       min_data_per_group = int(np.round(x[19])), 
                                       max_cat_threshold = int(np.round(x[20])), 
                                       cat_l2 = x[21],
                                       cat_smooth = x[22],
                                       max_cat_to_onehot = int(np.round(x[23])),
                                       top_k = int(np.round(x[24])),
                                       n_jobs = -1,
                                       )

        kfold = KFold(n_splits = 3, shuffle = True)
        
        scores = cross_val_score(model_lgb, x_fruit, y_fruit, cv = kfold, scoring='roc_auc_ovr_weighted', n_jobs=-1)  
        
        result = scores.mean()
        out['F'] = -1 * result

problemAUC = OptimizeWithAUC()

In [11]:
from pymoo.util.display.column import Column
from pymoo.util.display.output import Output

In [12]:
class MyOutput(Output):

    def __init__(self):
        super().__init__()
        global pbar 
        pbar = tqdm(total=ITERATIONS)
        self.score = Column("score", width=13)
        self.Parameters = Column("Parameters", width=35)
        self.columns += [self.score, self.Parameters]

    def update(self, algorithm):
        super().update(algorithm)
        self.score.set(-np.min(algorithm.pop.get("F")))
        #self.Parameters.set(algorithm.pop.get("X")[0])
        pbar.update(1)
        if pbar.n == ITERATIONS: pbar.close()

## Particle Swarm Optimization (PSO)

### Acurácia

In [27]:
ITERATIONS = 32
POPULATION = 32

In [28]:
def run_accuracy_pso(ITERATIONS = 32, POPULATION = 32):
    algorithm = PSO(#pop_size=POPULATION, 
                    #w=0.3, c1=4.0, c2=1.0, 
                    #adaptive=False, initial_velocity='random', 
                    #max_velocity_rate=0.75, pertube_best=False,
                    max_velocity_rate=0.25,
                    pop_size=POPULATION)
    term = get_termination("n_gen", ITERATIONS)

    res = minimize(problemAccuracy,
                algorithm,
                #seed=SEED,
                save_history=False,
                verbose=False,
                output=MyOutput(),
                termination = term)


    index_best_individual = np.where(res.pop.get('F') == np.min(res.pop.get('F')))[0][0]
    score_best_individual = res.pop.get('F')[index_best_individual]
    parameters_best_individual = res.pop.get('X')[index_best_individual]

    #print(f'Best Accuracy Score {-score_best_individual}')
    #print(f'Model parameters: \n {parameters_best_individual}')
    return score_best_individual, parameters_best_individual, res

In [29]:
score_best_individual, parameters_best_individual, res = run_accuracy_pso(ITERATIONS = 1, POPULATION = 1)

C:\Users\Felps\AppData\Local\Temp\ipykernel_16080\2673210061.py:8: DeprecationWarning: Call to deprecated function (or staticmethod) get_termination. (Please use `from pymoo.termination import get_termination`)
  term = get_termination("n_gen", ITERATIONS)


  0%|          | 0/32 [00:00<?, ?it/s]

In [30]:
#final_model = lgb.LGBMClassifier(num_leaves       = int(np.round(parameters_best_individual[0])), 
#                                min_child_samples = int(np.round(parameters_best_individual[1])), 
#                                n_estimators      = int(np.round(parameters_best_individual[2])),
#                                learning_rate     = parameters_best_individual[3] ,
#                                subsample_for_bin = int(np.round(parameters_best_individual[4])),
#                                min_split_gain    = parameters_best_individual[5],
#                                min_child_weight  = parameters_best_individual[6],
#                                reg_alpha         = parameters_best_individual[7],
#                                reg_lambda        = parameters_best_individual[8],
#                                max_depth=- 1
#                                )
#
#
#kfold = KFold(n_splits = 3, shuffle = True)
#
#scores = cross_val_score(final_model, x_fruit, y_fruit, cv = kfold, n_jobs=-1)  
#
#result = scores.mean()
###print(result)

In [31]:
#trlist = np.array([])
#for i in range(len(res.history)):
#  trlist = np.append(trlist, -res.history[i].pop.get('F').reshape(-1)) 

In [32]:
#scorelist = []
#
#for i in range(len(trlist)):
##for i in range(1):
#  if i == 0:
#    scorelist.append(trlist[i])
#  elif trlist[i] > scorelist[i-1]:
#    scorelist.append(trlist[i])
#  else:
#    scorelist.append(scorelist[i-1])

In [33]:
#Accuracy_PSO = scorelist

In [34]:
#plt.plot(Accuracy_PSO);

In [35]:
#tracking = [-np.min(individual.pop.get('F')) for individual in res.history ]
#tracking_PSO_Accuracy = tracking
#plt.plot(tracking)

### F1 Score 

In [36]:
def run_f1_pso(ITERATIONS = 32, POPULATION = 32):
    algorithm = PSO(#pop_size=POPULATION, 
                    #w=0.3, c1=4.0, c2=1.0, 
                    #adaptive=False, initial_velocity='random', 
                    #max_velocity_rate=0.75, pertube_best=False,
                    max_velocity_rate=0.25,
                    pop_size=POPULATION)
    term = get_termination("n_gen", ITERATIONS)

    res = minimize(problemF1,
                algorithm,
                #seed=SEED,
                save_history=False,
                verbose=False,
                output=MyOutput(),
                termination = term)


    index_best_individual = np.where(res.pop.get('F') == np.min(res.pop.get('F')))[0][0]
    score_best_individual = res.pop.get('F')[index_best_individual]
    parameters_best_individual = res.pop.get('X')[index_best_individual]

    #print(f'Best F1 Score {-score_best_individual}')
    #print(f'Model parameters: \n {parameters_best_individual}')
    return score_best_individual, parameters_best_individual, res

In [37]:
score_best_individual, parameters_best_individual, res = run_f1_pso(ITERATIONS = 1, POPULATION = 1)

C:\Users\Felps\AppData\Local\Temp\ipykernel_16080\3496873573.py:8: DeprecationWarning: Call to deprecated function (or staticmethod) get_termination. (Please use `from pymoo.termination import get_termination`)
  term = get_termination("n_gen", ITERATIONS)


  0%|          | 0/32 [00:00<?, ?it/s]

In [38]:
#trlist = np.array([])
#for i in range(len(res.history)):
#  trlist = np.append(trlist, -res.history[i].pop.get('F').reshape(-1)) 
#  
#scorelist = []
#
#for i in range(len(trlist)):
##for i in range(1):
#  if i == 0:
#    scorelist.append(trlist[i])
#  elif trlist[i] > scorelist[i-1]:
#    scorelist.append(trlist[i])
#  else:
#    scorelist.append(scorelist[i-1])
#    
#F1_PSO = scorelist
#
#plt.plot(scorelist);

In [39]:
#tracking = [-np.min(individual.pop.get('F')) for individual in res.history ]
#tracking_PSO_F1 = tracking
#plt.plot(tracking)

### AUC

In [40]:
def run_auc_pso(ITERATIONS = 32, POPULATION = 32):
    algorithm = PSO(#pop_size=POPULATION, 
                    #w=0.3, c1=4.0, c2=1.0, 
                    #adaptive=False, initial_velocity='random', 
                    #max_velocity_rate=0.75, pertube_best=False,
                    max_velocity_rate=0.25,
                    pop_size=POPULATION)
    term = get_termination("n_gen", ITERATIONS)

    res = minimize(problemAUC,
                algorithm,
                #seed=SEED,
                save_history=False,
                verbose=False,
                output=MyOutput(),
                termination = term)


    index_best_individual = np.where(res.pop.get('F') == np.min(res.pop.get('F')))[0][0]
    score_best_individual = res.pop.get('F')[index_best_individual]
    parameters_best_individual = res.pop.get('X')[index_best_individual]

    #print(f'Best AUC Score {-score_best_individual}')
    #print(f'Model parameters: \n {parameters_best_individual}')
    
    return score_best_individual, parameters_best_individual, res

In [41]:
score_best_individual, parameters_best_individual, res = run_auc_pso(ITERATIONS = 1, POPULATION = 1)

C:\Users\Felps\AppData\Local\Temp\ipykernel_16080\1707635262.py:8: DeprecationWarning: Call to deprecated function (or staticmethod) get_termination. (Please use `from pymoo.termination import get_termination`)
  term = get_termination("n_gen", ITERATIONS)


  0%|          | 0/32 [00:00<?, ?it/s]

In [42]:
#trlist = np.array([])
#for i in range(len(res.history)):
#  trlist = np.append(trlist, -res.history[i].pop.get('F').reshape(-1)) 
#  
#scorelist = []
#
#for i in range(len(trlist)):
##for i in range(1):
#  if i == 0:
#    scorelist.append(trlist[i])
#  elif trlist[i] > scorelist[i-1]:
#    scorelist.append(trlist[i])
#  else:
#    scorelist.append(scorelist[i-1])
#    
#AUC_PSO = scorelist
#
#plt.plot(scorelist);

In [43]:
#tracking = [-np.min(individual.pop.get('F')) for individual in res.history ]
#tracking_PSO_AUC = tracking
#plt.plot(tracking)

## Algoritmo Genético (GA)

### Acurácia

In [44]:
ITERATIONS = 32
POPULATION = 32

In [45]:
from pymoo.algorithms.soo.nonconvex.ga import GA

In [46]:
def run_accuracy_ga(ITERATIONS = 32, POPULATION = 32):

    algorithm = GA(pop_size=POPULATION)

    term = get_termination("n_gen", ITERATIONS)

    res = minimize(problemAccuracy,
                algorithm,
                save_history=False,
                verbose=False,
                output=MyOutput(),
                termination = term)


    index_best_individual = np.where(res.pop.get('F') == np.min(res.pop.get('F')))[0][0]
    score_best_individual = res.pop.get('F')[index_best_individual]
    parameters_best_individual = res.pop.get('X')[index_best_individual]

    #print(f'Best Accuracy Score {-score_best_individual}')
    #print(f'Model parameters: \n {parameters_best_individual}')
    
    return score_best_individual, parameters_best_individual, res

In [47]:
score_best_individual, parameters_best_individual, res = run_accuracy_ga(ITERATIONS = 1, POPULATION = 1)

C:\Users\Felps\AppData\Local\Temp\ipykernel_16080\3610728493.py:5: DeprecationWarning: Call to deprecated function (or staticmethod) get_termination. (Please use `from pymoo.termination import get_termination`)
  term = get_termination("n_gen", ITERATIONS)


  0%|          | 0/32 [00:00<?, ?it/s]

In [48]:
#tracking = [-np.min(individual.pop.get('F')) for individual in res.history ]
#tracking_GA_Accuracy = tracking
#plt.plot(tracking)

In [49]:
#trlist = np.array([])
#for i in range(len(res.history)):
#  trlist = np.append(trlist, -res.history[i].pop.get('F').reshape(-1)) 

In [50]:
#scorelist = []
#
#for i in range(len(trlist)):
#  if i == 0:
#    scorelist.append(trlist[i])
#  elif trlist[i] > scorelist[i-1]:
#    scorelist.append(trlist[i])
#  else:
#    scorelist.append(scorelist[i-1])

In [51]:
#Accuracy_GA = scorelist

In [52]:
#plt.plot(scorelist);

### F1 Score 

In [53]:
from pymoo.algorithms.soo.nonconvex.ga import GA
def run_f1_ga(ITERATIONS = 32, POPULATION = 32):
    algorithm = GA(pop_size=POPULATION)

    term = get_termination("n_gen", ITERATIONS)

    res = minimize(problemF1,
                algorithm,
                save_history=False,
                verbose=False,
                output=MyOutput(),
                termination = term)


    index_best_individual = np.where(res.pop.get('F') == np.min(res.pop.get('F')))[0][0]
    score_best_individual = res.pop.get('F')[index_best_individual]
    parameters_best_individual = res.pop.get('X')[index_best_individual]

    #print(f'Best F1 Score {-score_best_individual}')
    #print(f'Model parameters: \n {parameters_best_individual}')
    
    return score_best_individual, parameters_best_individual, res

In [54]:
score_best_individual, parameters_best_individual, res = run_f1_ga(ITERATIONS = 1, POPULATION = 1)

C:\Users\Felps\AppData\Local\Temp\ipykernel_16080\1994123235.py:5: DeprecationWarning: Call to deprecated function (or staticmethod) get_termination. (Please use `from pymoo.termination import get_termination`)
  term = get_termination("n_gen", ITERATIONS)


  0%|          | 0/32 [00:00<?, ?it/s]

In [55]:
#trlist = np.array([])
#for i in range(len(res.history)):
#  trlist = np.append(trlist, -res.history[i].pop.get('F').reshape(-1)) 
#  
#scorelist = []
#
#for i in range(len(trlist)):
##for i in range(1):
#  if i == 0:
#    scorelist.append(trlist[i])
#  elif trlist[i] > scorelist[i-1]:
#    scorelist.append(trlist[i])
#  else:
#    scorelist.append(scorelist[i-1])
#    
#F1_GA = scorelist
#
#plt.plot(scorelist);

In [56]:
#tracking = [-np.min(individual.pop.get('F')) for individual in res.history ]
#tracking_GA_F1 = tracking
#plt.plot(tracking)

### AUC

In [57]:
from pymoo.algorithms.soo.nonconvex.ga import GA
def run_auc_ga(ITERATIONS = 32, POPULATION = 32):
    algorithm = GA(pop_size=POPULATION)

    term = get_termination("n_gen", ITERATIONS)

    res = minimize(problemAUC,
                algorithm,
                save_history=False,
                verbose=False,
                output=MyOutput(),
                termination = term)


    index_best_individual = np.where(res.pop.get('F') == np.min(res.pop.get('F')))[0][0]
    score_best_individual = res.pop.get('F')[index_best_individual]
    parameters_best_individual = res.pop.get('X')[index_best_individual]

    #print(f'Best AUC Score {-score_best_individual}')
    #print(f'Model parameters: \n {parameters_best_individual}')
    
    return score_best_individual, parameters_best_individual, res

In [58]:
score_best_individual, parameters_best_individual, res = run_auc_ga(ITERATIONS = 1, POPULATION = 1)

C:\Users\Felps\AppData\Local\Temp\ipykernel_16080\3250583479.py:5: DeprecationWarning: Call to deprecated function (or staticmethod) get_termination. (Please use `from pymoo.termination import get_termination`)
  term = get_termination("n_gen", ITERATIONS)


  0%|          | 0/32 [00:00<?, ?it/s]

In [59]:
#trlist = np.array([])
#for i in range(len(res.history)):
#  trlist = np.append(trlist, -res.history[i].pop.get('F').reshape(-1)) 
#  
#scorelist = []
#
#for i in range(len(trlist)):
##for i in range(1):
#  if i == 0:
#    scorelist.append(trlist[i])
#  elif trlist[i] > scorelist[i-1]:
#    scorelist.append(trlist[i])
#  else:
#    scorelist.append(scorelist[i-1])
#    
#AUC_GA = scorelist
#
#plt.plot(scorelist);

In [60]:
#tracking = [-np.min(individual.pop.get('F')) for individual in res.history ]
#tracking_GA_AUC = tracking
#plt.plot(tracking)

## Grid Search

### Acurácia

In [61]:
from sklearn.model_selection import GridSearchCV

In [62]:
#n_possibilities = 2
#
##num_leaves_grid = [i for i in range(num_leaves_min,num_leaves_max, int((num_leaves_max)/13))]
#num_leaves_grid = [i for i in map(lambda x: int(x), np.linspace(num_leaves_min, num_leaves_max, n_possibilities))]
#num_leaves_grid = num_leaves_grid + [100, 50, 75, 125, 11,150]
##print('Num_Leaves_Grid: ')
##print(num_leaves_grid, len(num_leaves_grid))
##print('\n')
#
#
#min_child_samples_grid = [i for i in map(lambda x: int(x), np.linspace(min_child_samples_min, min_child_samples_max, n_possibilities))]
##print('min_child_samples_grid:')
##print(min_child_samples_grid, len(min_child_samples_grid))
##print('\n')
#
#n_estimators_grid = [i for i in map(lambda x: int(x), np.linspace(n_estimators_min, n_estimators_max, n_possibilities))]
##print('n_estimators_grid:')
##print(n_estimators_grid, len(n_estimators_grid))
##print('\n')
#
#learning_rate_grid = np.linspace(learning_rate_min, learning_rate_max, n_possibilities)
##print('learning_rate_grid:')
##print(learning_rate_grid, len(learning_rate_grid))
##print('\n')
#
#subsample_for_bin_grid = [i for i in map(lambda x: int(x), np.linspace(subsample_for_bin_min, subsample_for_bin_max, n_possibilities))]
##print('subsample_for_bin_grid:')
##print(subsample_for_bin_grid, len(subsample_for_bin_grid))
##print('\n')
#
#min_split_gain_grid = np.linspace(min_split_gain_min, min_split_gain_max, n_possibilities)
##print('min_split_gain_grid:')
##print(min_split_gain_grid, len(min_split_gain_grid))
##print('\n')
#
#min_child_weight_grid = np.linspace(min_child_weight_min, min_child_weight_max, n_possibilities)
##print('min_child_weight_grid:')
##print(min_child_weight_grid, len(min_child_weight_grid))
##print('\n')
#
#reg_alpha_grid = np.linspace(reg_alpha_min, reg_alpha_max, n_possibilities)
##print('reg_alpha_grid:')
##print(reg_alpha_grid, len(reg_alpha_grid))
##print('\n')


In [63]:
#parametros = {'num_leaves': num_leaves_grid, #int
#              'min_child_samples': min_child_samples_grid,#int
#              'n_estimators': n_estimators_grid, #int
#              'learning_rate': learning_rate_grid,
#              'subsample_for_bin': subsample_for_bin_grid, # int
#              'min_split_gain': min_split_gain_grid,
#              'min_child_weight': min_child_weight_grid,
#              'reg_alpha': reg_alpha_grid,
#              'max_depth': [-1],
#              'n_jobs': [-1]}

In [64]:
#kfold = KFold(n_splits = 3, shuffle = True)
#grid_search = GridSearchCV(estimator = lgb.LGBMClassifier(), param_grid = parametros, cv = kfold, n_jobs= -1, verbose = 3)
#grid_search.fit(x_fruit, y_fruit)
#melhores_parametros = grid_search.best_params_
#melhor_resultado = grid_search.best_score_
##print(melhores_parametros)
##print(melhor_resultado)

In [65]:
#grid_search.cv_results_.keys()

In [66]:
#trlist = grid_search.cv_results_['mean_test_score']

In [67]:
#scorelist = []
#
#for i in range(len(trlist)):
##for i in range(1):
#  if i == 0:
#    scorelist.append(trlist[i])
#  elif trlist[i] > scorelist[i-1]:
#    scorelist.append(trlist[i])
#  else:
#    scorelist.append(scorelist[i-1])

In [68]:
#tracking_GS_Accuracy = scorelist
#plt.plot(scorelist); 

### F1 Score

In [69]:
#kfold = KFold(n_splits = 3, shuffle = True)
#grid_search = GridSearchCV(estimator = lgb.LGBMClassifier(), param_grid = parametros, cv = kfold, n_jobs= -1, scoring='f1', verbose = 3)
#grid_search.fit(x_fruit, y_fruit)
#melhores_parametros = grid_search.best_params_
#melhor_resultado = grid_search.best_score_
##print(melhores_parametros)
##print(melhor_resultado)
#
#trlist = grid_search.cv_results_['mean_test_score']
#
#
#scorelist = []
#
#for i in range(len(trlist)):
##for i in range(1):
#  if i == 0:
#    scorelist.append(trlist[i])
#  elif trlist[i] > scorelist[i-1]:
#    scorelist.append(trlist[i])
#  else:
#    scorelist.append(scorelist[i-1])
#    
#tracking_GS_F1 = scorelist
#plt.plot(scorelist); 

### AUC

In [70]:
#kfold = KFold(n_splits = 3, shuffle = True)
#grid_search = GridSearchCV(estimator = lgb.LGBMClassifier(), param_grid = parametros, cv = kfold, n_jobs= -1, scoring='roc_auc', verbose = 3)
#grid_search.fit(x_fruit, y_fruit)
#melhores_parametros = grid_search.best_params_
#melhor_resultado = grid_search.best_score_
##print(melhores_parametros)
##print(melhor_resultado)
#
#trlist = grid_search.cv_results_['mean_test_score']
#
#
#scorelist = []
#
#for i in range(len(trlist)):
##for i in range(1):
#  if i == 0:
#    scorelist.append(trlist[i])
#  elif trlist[i] > scorelist[i-1]:
#    scorelist.append(trlist[i])
#  else:
#    scorelist.append(scorelist[i-1])
#    
#tracking_GS_AUC = scorelist
#plt.plot(scorelist); 

## Optuna

### Acurácia

In [71]:
#!pip install optuna

In [72]:
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)
import sklearn
from sklearn import datasets
def objective_accuracy(trial):
  
      num_leaves = trial.suggest_int('num_leaves', num_leaves_min, num_leaves_max) #int
      min_child_samples = trial.suggest_int('min_child_samples', min_child_samples_min, min_child_samples_max) #int
      n_estimators      = trial.suggest_int('n_estimators', n_estimators_min, n_estimators_max)#int
      learning_rate     = trial.suggest_float('learning_rate', learning_rate_min, learning_rate_max)
      subsample_for_bin = trial.suggest_int('subsample_for_bin', subsample_for_bin_min, subsample_for_bin_max) #int
      min_split_gain    = trial.suggest_float('min_split_gain', min_split_gain_min, min_split_gain_max)
      min_child_weight  = trial.suggest_float('min_child_weight', min_child_weight_min, min_child_weight_max)
      reg_alpha         = trial.suggest_float('reg_alpha', reg_alpha_min, reg_alpha_max)
      reg_lambda        = trial.suggest_float('reg_lambda', reg_lambda_min, reg_lambda_max)
      max_depth = trial.suggest_int('max_depth', max_depth_min, max_depth_max)
      bagging_fraction = trial.suggest_float('bagging_fraction', bagging_fraction_min, bagging_fraction_max)
      pos_bagging_fraction = trial.suggest_float('pos_bagging_fraction', pos_bagging_fraction_min, pos_bagging_fraction_max)
      neg_bagging_fraction = trial.suggest_float('neg_bagging_fraction', neg_bagging_fraction_min, neg_bagging_fraction_max)
      bagging_freq = trial.suggest_int('bagging_freq', bagging_freq_min, bagging_freq_max)
      feature_fraction = trial.suggest_float('feature_fraction', feature_fraction_min, feature_fraction_max)
      feature_fraction_bynode = trial.suggest_float('feature_fraction_bynode', feature_fraction_bynode_min, feature_fraction_bynode_max)
      extra_trees = trial.suggest_int('extra_trees', extra_trees_min, extra_trees_max)
      #first_metric_only = trial.suggest_int('first_metric_only', first_metric_only_min, first_metric_only_max)
      max_delta_step = trial.suggest_float('max_delta_step', max_delta_step_min, max_delta_step_max)
      linear_lambda = trial.suggest_float('linear_lambda', linear_lambda_min, linear_lambda_max)
      min_data_per_group = trial.suggest_int('min_data_per_group', min_data_per_group_min, min_data_per_group_max)
      max_cat_threshold = trial.suggest_int('max_cat_threshold', max_cat_threshold_min, max_cat_threshold_max)
      cat_l2 = trial.suggest_float('cat_l2', cat_l2_min, cat_l2_max)
      cat_smooth = trial.suggest_float('cat_smooth', cat_smooth_min, cat_smooth_max)
      max_cat_to_onehot = trial.suggest_int('max_cat_to_onehot', max_cat_to_onehot_min, max_cat_to_onehot_max)
      top_k = trial.suggest_int('top_k', top_k_min, top_k_max)

      final_model = lgb.LGBMClassifier(num_leaves       = num_leaves, 
                                      min_child_samples = min_child_samples, 
                                      n_estimators      = n_estimators,
                                      learning_rate     = learning_rate,
                                      subsample_for_bin = subsample_for_bin,
                                      min_split_gain    = min_split_gain,
                                      min_child_weight  = min_child_weight,
                                      reg_alpha         = reg_alpha,
                                      reg_lambda        = reg_lambda,
                                      max_depth= -1,
                                      n_jobs = -1
                                      )
      
      kfold = KFold(n_splits = 3, shuffle = True)

      return sklearn.model_selection.cross_val_score(final_model, x_fruit, y_fruit, n_jobs=-1, cv=kfold).mean()

In [73]:
def run_optuna_accuracy(n_trials=1024):
    study = optuna.create_study(direction='maximize')
    study.optimize(objective_accuracy, n_trials=n_trials, n_jobs = -1)
    trial = study.best_trial
    #print('Accuracy: {}'.format(trial.value))
    #print("Best hyperparameters: {}".format(trial.params))
    return trial.value, study


In [74]:
trial, study = run_optuna_accuracy(1)

In [75]:
trial

0.8563582311408399

In [76]:
#tracking = []
#for i in range(len(study.trials)):
#  if i == 0: 
#    tracking.append(study.trials[i].value)
#  elif tracking[i-1] > study.trials[i].value:
#    tracking.append(tracking[i-1])
#  else:
#    tracking.append(study.trials[i].value)
##tracking_sorted = sorted(tracking)

In [77]:
#tracking_Optuna_Accuracy = tracking

In [78]:
#plt.plot(tracking);
#plt.plot(tracking_sorted) 

In [79]:
#optuna.visualization.plot_optimization_history(study)

In [80]:
#optuna.visualization.plot_slice(study)

### F1 Score

In [81]:
def objective_f1(trial):
  
      num_leaves = trial.suggest_int('num_leaves', num_leaves_min, num_leaves_max) #int
      min_child_samples = trial.suggest_int('min_child_samples', min_child_samples_min, min_child_samples_max) #int
      n_estimators      = trial.suggest_int('n_estimators', n_estimators_min, n_estimators_max)#int
      learning_rate     = trial.suggest_float('learning_rate', learning_rate_min, learning_rate_max)
      subsample_for_bin = trial.suggest_int('subsample_for_bin', subsample_for_bin_min, subsample_for_bin_max) #int
      min_split_gain    = trial.suggest_float('min_split_gain', min_split_gain_min, min_split_gain_max)
      min_child_weight  = trial.suggest_float('min_child_weight', min_child_weight_min, min_child_weight_max)
      reg_alpha         = trial.suggest_float('reg_alpha', reg_alpha_min, reg_alpha_max)
      reg_lambda        = trial.suggest_float('reg_lambda', reg_lambda_min, reg_lambda_max)
      max_depth = trial.suggest_int('max_depth', max_depth_min, max_depth_max)
      bagging_fraction = trial.suggest_float('bagging_fraction', bagging_fraction_min, bagging_fraction_max)
      pos_bagging_fraction = trial.suggest_float('pos_bagging_fraction', pos_bagging_fraction_min, pos_bagging_fraction_max)
      neg_bagging_fraction = trial.suggest_float('neg_bagging_fraction', neg_bagging_fraction_min, neg_bagging_fraction_max)
      bagging_freq = trial.suggest_int('bagging_freq', bagging_freq_min, bagging_freq_max)
      feature_fraction = trial.suggest_float('feature_fraction', feature_fraction_min, feature_fraction_max)
      feature_fraction_bynode = trial.suggest_float('feature_fraction_bynode', feature_fraction_bynode_min, feature_fraction_bynode_max)
      extra_trees = trial.suggest_int('extra_trees', extra_trees_min, extra_trees_max)
      #first_metric_only = trial.suggest_int('first_metric_only', first_metric_only_min, first_metric_only_max)
      max_delta_step = trial.suggest_float('max_delta_step', max_delta_step_min, max_delta_step_max)
      linear_lambda = trial.suggest_float('linear_lambda', linear_lambda_min, linear_lambda_max)
      min_data_per_group = trial.suggest_int('min_data_per_group', min_data_per_group_min, min_data_per_group_max)
      max_cat_threshold = trial.suggest_int('max_cat_threshold', max_cat_threshold_min, max_cat_threshold_max)
      cat_l2 = trial.suggest_float('cat_l2', cat_l2_min, cat_l2_max)
      cat_smooth = trial.suggest_float('cat_smooth', cat_smooth_min, cat_smooth_max)
      max_cat_to_onehot = trial.suggest_int('max_cat_to_onehot', max_cat_to_onehot_min, max_cat_to_onehot_max)
      top_k = trial.suggest_int('top_k', top_k_min, top_k_max)

      final_model = lgb.LGBMClassifier(num_leaves       = num_leaves, 
                                      min_child_samples = min_child_samples, 
                                      n_estimators      = n_estimators,
                                      learning_rate     = learning_rate,
                                      subsample_for_bin = subsample_for_bin,
                                      min_split_gain    = min_split_gain,
                                      min_child_weight  = min_child_weight,
                                      reg_alpha         = reg_alpha,
                                      reg_lambda        = reg_lambda,
                                      max_depth= -1,
                                      n_jobs = -1
                                      )
      
      kfold = KFold(n_splits = 3, shuffle = True)

      return sklearn.model_selection.cross_val_score(final_model, x_fruit, y_fruit, n_jobs=-1, scoring='f1_weighted', cv=kfold).mean()

In [82]:
def run_optuna_f1(n_trials=1024):
    study = optuna.create_study(direction='maximize')
    study.optimize(objective_f1, n_trials=n_trials, n_jobs = -1)
    trial = study.best_trial
    #print('F1: {}'.format(trial.value))
    #print("Best hyperparameters: {}".format(trial.params))
    return trial.value, study

In [83]:
trial, study = run_optuna_f1(1)

In [84]:
#tracking = []
#for i in range(len(study.trials)):
#  if i == 0: 
#    tracking.append(study.trials[i].value)
#  elif tracking[i-1] > study.trials[i].value:
#    tracking.append(tracking[i-1])
#  else:
#    tracking.append(study.trials[i].value)
##tracking_sorted = sorted(tracking)
#
#tracking_Optuna_F1 = tracking
#
#plt.plot(tracking);
##plt.plot(tracking_sorted) 

### AUC

In [85]:
def objective_auc(trial):
  
      num_leaves = trial.suggest_int('num_leaves', num_leaves_min, num_leaves_max) #int
      min_child_samples = trial.suggest_int('min_child_samples', min_child_samples_min, min_child_samples_max) #int
      n_estimators      = trial.suggest_int('n_estimators', n_estimators_min, n_estimators_max)#int
      learning_rate     = trial.suggest_float('learning_rate', learning_rate_min, learning_rate_max)
      subsample_for_bin = trial.suggest_int('subsample_for_bin', subsample_for_bin_min, subsample_for_bin_max) #int
      min_split_gain    = trial.suggest_float('min_split_gain', min_split_gain_min, min_split_gain_max)
      min_child_weight  = trial.suggest_float('min_child_weight', min_child_weight_min, min_child_weight_max)
      reg_alpha         = trial.suggest_float('reg_alpha', reg_alpha_min, reg_alpha_max)
      reg_lambda        = trial.suggest_float('reg_lambda', reg_lambda_min, reg_lambda_max)
      max_depth = trial.suggest_int('max_depth', max_depth_min, max_depth_max)
      bagging_fraction = trial.suggest_float('bagging_fraction', bagging_fraction_min, bagging_fraction_max)
      pos_bagging_fraction = trial.suggest_float('pos_bagging_fraction', pos_bagging_fraction_min, pos_bagging_fraction_max)
      neg_bagging_fraction = trial.suggest_float('neg_bagging_fraction', neg_bagging_fraction_min, neg_bagging_fraction_max)
      bagging_freq = trial.suggest_int('bagging_freq', bagging_freq_min, bagging_freq_max)
      feature_fraction = trial.suggest_float('feature_fraction', feature_fraction_min, feature_fraction_max)
      feature_fraction_bynode = trial.suggest_float('feature_fraction_bynode', feature_fraction_bynode_min, feature_fraction_bynode_max)
      extra_trees = trial.suggest_int('extra_trees', extra_trees_min, extra_trees_max)
      #first_metric_only = trial.suggest_int('first_metric_only', first_metric_only_min, first_metric_only_max)
      max_delta_step = trial.suggest_float('max_delta_step', max_delta_step_min, max_delta_step_max)
      linear_lambda = trial.suggest_float('linear_lambda', linear_lambda_min, linear_lambda_max)
      min_data_per_group = trial.suggest_int('min_data_per_group', min_data_per_group_min, min_data_per_group_max)
      max_cat_threshold = trial.suggest_int('max_cat_threshold', max_cat_threshold_min, max_cat_threshold_max)
      cat_l2 = trial.suggest_float('cat_l2', cat_l2_min, cat_l2_max)
      cat_smooth = trial.suggest_float('cat_smooth', cat_smooth_min, cat_smooth_max)
      max_cat_to_onehot = trial.suggest_int('max_cat_to_onehot', max_cat_to_onehot_min, max_cat_to_onehot_max)
      top_k = trial.suggest_int('top_k', top_k_min, top_k_max)

      final_model = lgb.LGBMClassifier(num_leaves       = num_leaves, 
                                      min_child_samples = min_child_samples, 
                                      n_estimators      = n_estimators,
                                      learning_rate     = learning_rate,
                                      subsample_for_bin = subsample_for_bin,
                                      min_split_gain    = min_split_gain,
                                      min_child_weight  = min_child_weight,
                                      reg_alpha         = reg_alpha,
                                      reg_lambda        = reg_lambda,
                                      max_depth= -1,
                                      n_jobs = -1
                                      )
      
      kfold = KFold(n_splits = 3, shuffle = True)

      return sklearn.model_selection.cross_val_score(final_model, x_fruit, y_fruit, n_jobs=-1, scoring='roc_auc_ovr_weighted', cv=kfold).mean()

In [86]:
def run_optuna_auc(n_trials=1024):
    study = optuna.create_study(direction='maximize')
    study.optimize(objective_auc, n_trials=n_trials, n_jobs = -1)
    trial = study.best_trial
    #print('AUC: {}'.format(trial.value))
    #print("Best hyperparameters: {}".format(trial.params))
    return trial.value, study

In [87]:
trial, study = run_optuna_auc(1)

In [88]:
#tracking = []
#for i in range(len(study.trials)):
#  if i == 0: 
#    tracking.append(study.trials[i].value)
#  elif tracking[i-1] > study.trials[i].value:
#    tracking.append(tracking[i-1])
#  else:
#    tracking.append(study.trials[i].value)
##tracking_sorted = sorted(tracking)
#
#tracking_Optuna_AUC = tracking
#
#plt.plot(tracking);
##plt.plot(tracking_sorted) 

# Análise Comparativa

In [89]:
#fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,5))
#
#ax1.set_title('PSO X Gerações')
#ax2.set_title('PSO X Avaliações')
#
#ax1.plot(tracking_PSO_Accuracy)
#ax2.plot(Accuracy_PSO)
#plt.show()

In [90]:
#fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,5))
#
#ax1.set_title('GA X Gerações')
#ax2.set_title('GA X Avaliações')
#
#ax1.plot(tracking_GA_Accuracy)
#ax2.plot(Accuracy_GA)
#plt.show()

Gráficos Comparativos Acurácia

In [91]:
#from IPython.core.pylabtools import figsize
#from matplotlib.pyplot import figure
#import matplotlib.patches as mpatches
#
#plt.figure(figsize=(12, 6))
#red_patch = mpatches.Patch(color='red', label='PSO')
#blue_patch = mpatches.Patch(color='blue', label='GA')
#green_patch = mpatches.Patch(color='green', label='Optuna')
##black_patch = mpatches.Patch(color='black', label='GridSearch')
#
#plt.legend(handles=[red_patch, blue_patch, green_patch])
#plt.plot(Accuracy_GA, color = 'blue', linewidth=1, linestyle='-', )
#plt.plot(Accuracy_PSO, color = 'red', linewidth=1, linestyle='-')
#plt.plot(tracking_Optuna_Accuracy, color = 'green', linewidth=1, linestyle='-')
##plt.plot(tracking_GS, color = 'black', linewidth=3, linestyle='-')
#plt.show()

In [92]:
#fig, ax = plt.subplots(2, 2, figsize=(25,15))
#
#ax[0, 0].set_title(f'Genetic Algorithm (GA) - Accuracy: {max(tracking_GA_Accuracy)}', fontdict={'fontsize': 20, 'fontweight': 'medium'})
#ax[0, 1].set_title(f'Particle Swarm Optimization (PSO) - Accuracy: {max(tracking_PSO_Accuracy)}', fontdict={'fontsize': 20, 'fontweight': 'medium'})
#
#ax[1, 0].set_title(f'Grid Search - Accuracy: {max(tracking_GS_Accuracy)}', fontdict={'fontsize': 20, 'fontweight': 'medium'})
#ax[1, 1].set_title(f'Optuna - Accuracy: {max(tracking_Optuna_Accuracy)}', fontdict={'fontsize': 20, 'fontweight': 'medium'})
#
#ax[0,0].plot(tracking_GA_Accuracy, color = 'blue', linewidth=3, linestyle='-', )
#ax[0,1].plot(tracking_PSO_Accuracy, color = 'red', linewidth=3, linestyle='-')
#
#ax[1,0].plot(tracking_GS_Accuracy, color = 'black', linewidth=3, linestyle='-')
#ax[1,1].plot(tracking_Optuna_Accuracy, color = 'green', linewidth=3, linestyle='-')


F1 score

In [93]:
#plt.figure(figsize=(12, 6))
#red_patch = mpatches.Patch(color='red', label='PSO')
#blue_patch = mpatches.Patch(color='blue', label='GA')
#green_patch = mpatches.Patch(color='green', label='Optuna')
##black_patch = mpatches.Patch(color='black', label='GridSearch')
#
#plt.legend(handles=[red_patch, blue_patch, green_patch])
#plt.plot(F1_GA, color = 'blue', linewidth=1, linestyle='-', )
#plt.plot(F1_PSO, color = 'red', linewidth=1, linestyle='-')
#plt.plot(tracking_Optuna_F1, color = 'green', linewidth=1, linestyle='-')
##plt.plot(tracking_GS, color = 'black', linewidth=3, linestyle='-')
#plt.show()

In [94]:
#fig, ax = plt.subplots(2, 2, figsize=(25,15))
#
#ax[0, 0].set_title(f'Genetic Algorithm (GA) - Accuracy: {max(tracking_GA_F1)}', fontdict={'fontsize': 20, 'fontweight': 'medium'})
#ax[0, 1].set_title(f'Particle Swarm Optimization (PSO) - Accuracy: {max(tracking_PSO_F1)}', fontdict={'fontsize': 20, 'fontweight': 'medium'})
#
#ax[1, 0].set_title(f'Grid Search - Accuracy: {max(tracking_GS_F1)}', fontdict={'fontsize': 20, 'fontweight': 'medium'})
#ax[1, 1].set_title(f'Optuna - Accuracy: {max(tracking_Optuna_F1)}', fontdict={'fontsize': 20, 'fontweight': 'medium'})
#
#ax[0,0].plot(tracking_GA_F1, color = 'blue', linewidth=3, linestyle='-', )
#ax[0,1].plot(tracking_PSO_F1, color = 'red', linewidth=3, linestyle='-')
#
#ax[1,0].plot(tracking_GS_F1, color = 'black', linewidth=3, linestyle='-')
#ax[1,1].plot(tracking_Optuna_F1, color = 'green', linewidth=3, linestyle='-')


ROC AUC

In [95]:
#plt.figure(figsize=(12, 6))
#red_patch = mpatches.Patch(color='red', label='PSO')
#blue_patch = mpatches.Patch(color='blue', label='GA')
#green_patch = mpatches.Patch(color='green', label='Optuna')
##black_patch = mpatches.Patch(color='black', label='GridSearch')
#
#plt.legend(handles=[red_patch, blue_patch, green_patch])
#plt.plot(AUC_GA, color = 'blue', linewidth=1, linestyle='-', )
#plt.plot(AUC_PSO, color = 'red', linewidth=1, linestyle='-')
#plt.plot(tracking_Optuna_AUC, color = 'green', linewidth=1, linestyle='-')
##plt.plot(tracking_GS, color = 'black', linewidth=3, linestyle='-')
#plt.show()

In [13]:
#fig, ax = plt.subplots(2, 2, figsize=(25,15))
#
#ax[0, 0].set_title(f'Genetic Algorithm (GA) - Accuracy: {max(tracking_GA_AUC)}', fontdict={'fontsize': 20, 'fontweight': 'medium'})
#ax[0, 1].set_title(f'Particle Swarm Optimization (PSO) - Accuracy: {max(tracking_PSO_AUC)}', fontdict={'fontsize': 20, 'fontweight': 'medium'})
#
#ax[1, 0].set_title(f'Grid Search - Accuracy: {max(tracking_GS_AUC)}', fontdict={'fontsize': 20, 'fontweight': 'medium'})
#ax[1, 1].set_title(f'Optuna - Accuracy: {max(tracking_Optuna_AUC)}', fontdict={'fontsize': 20, 'fontweight': 'medium'})
#
#ax[0,0].plot(tracking_GA_AUC, color = 'blue', linewidth=3, linestyle='-', )
#ax[0,1].plot(tracking_PSO_AUC, color = 'red', linewidth=3, linestyle='-')
#
#ax[1,0].plot(tracking_GS_AUC, color = 'black', linewidth=3, linestyle='-')
#ax[1,1].plot(tracking_Optuna_AUC, color = 'green', linewidth=3, linestyle='-')
#

In [14]:
from scipy.stats import uniform
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV

def random_search_int_range(min_l, max_l):
    return [int(i) for i in np.arange(randint.ppf(0.01, min_l, max_l + 2),randint.ppf(0.99, min_l, max_l + 2))]


def run_random_search_accuracy(iterations = 1024):
    param_distribution = {
        'num_leaves' : random_search_int_range(num_leaves_min, num_leaves_max),
        'min_child_samples' : random_search_int_range(min_child_samples_min, min_child_samples_max),
        'n_estimators' : random_search_int_range(n_estimators_min, n_estimators_max),
        'learning_rate' : uniform(learning_rate_min, learning_rate_max),
        'subsample_for_bin' : random_search_int_range(subsample_for_bin_min, subsample_for_bin_max),
        'min_split_gain' : uniform(min_split_gain_min, min_split_gain_max),
        'min_child_weight' : uniform(min_child_weight_min, min_child_weight_max),
        'reg_alpha' : uniform(reg_alpha_min, reg_alpha_max),
        'reg_lambda' : uniform(reg_lambda_min, reg_lambda_max),
        'max_depth' : random_search_int_range(max_depth_min, max_depth_max),
        'bagging_fraction' : uniform(bagging_fraction_min, bagging_fraction_max),
        'pos_bagging_fraction' : uniform(pos_bagging_fraction_min, pos_bagging_fraction_max),
        'neg_bagging_fraction' : uniform(neg_bagging_fraction_min, neg_bagging_fraction_max),
        'bagging_freq' : random_search_int_range(bagging_freq_min, bagging_freq_max),
        'feature_fraction' : uniform(feature_fraction_min, feature_fraction_max),
        'feature_fraction_bynode' : uniform(feature_fraction_bynode_min, feature_fraction_bynode_max),
        'extra_trees' : [True, False],
        'max_delta_step' : uniform(max_delta_step_min, max_delta_step_max),
        'linear_lambda' : uniform(linear_lambda_min, linear_lambda_max),
        'min_data_per_group' : random_search_int_range(min_data_per_group_min, min_data_per_group_max),
        'max_cat_threshold' : random_search_int_range(max_cat_threshold_min, max_cat_threshold_max),
        'cat_l2' : uniform(cat_l2_min, cat_l2_max),
        'cat_smooth' : uniform(cat_smooth_min, cat_smooth_max),
        'max_cat_to_onehot' : random_search_int_range(max_cat_to_onehot_min, max_cat_to_onehot_max),
        'top_k' : random_search_int_range(top_k_min, top_k_max),
        'n_jobs' : [-1]
    }




    kfold = KFold(n_splits = 3, shuffle = True)

    random_search_accuracy = RandomizedSearchCV(lgb.LGBMClassifier(), param_distribution, n_iter = iterations, n_jobs = 10, cv=kfold, verbose=False)
    random_search_accuracy.fit(x_fruit, y_fruit)

    return random_search_accuracy.best_score_

In [15]:
from scipy.stats import uniform
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV

def random_search_int_range(min_l, max_l):
    return [int(i) for i in np.arange(randint.ppf(0.01, min_l, max_l + 2),randint.ppf(0.99, min_l, max_l + 2))]


def run_random_search_f1(iterations = 1024):
    param_distribution = {
        'num_leaves' : random_search_int_range(num_leaves_min, num_leaves_max),
        'min_child_samples' : random_search_int_range(min_child_samples_min, min_child_samples_max),
        'n_estimators' : random_search_int_range(n_estimators_min, n_estimators_max),
        'learning_rate' : uniform(learning_rate_min, learning_rate_max),
        'subsample_for_bin' : random_search_int_range(subsample_for_bin_min, subsample_for_bin_max),
        'min_split_gain' : uniform(min_split_gain_min, min_split_gain_max),
        'min_child_weight' : uniform(min_child_weight_min, min_child_weight_max),
        'reg_alpha' : uniform(reg_alpha_min, reg_alpha_max),
        'reg_lambda' : uniform(reg_lambda_min, reg_lambda_max),
        'max_depth' : random_search_int_range(max_depth_min, max_depth_max),
        'bagging_fraction' : uniform(bagging_fraction_min, bagging_fraction_max),
        'pos_bagging_fraction' : uniform(pos_bagging_fraction_min, pos_bagging_fraction_max),
        'neg_bagging_fraction' : uniform(neg_bagging_fraction_min, neg_bagging_fraction_max),
        'bagging_freq' : random_search_int_range(bagging_freq_min, bagging_freq_max),
        'feature_fraction' : uniform(feature_fraction_min, feature_fraction_max),
        'feature_fraction_bynode' : uniform(feature_fraction_bynode_min, feature_fraction_bynode_max),
        'extra_trees' : [True, False],
        'max_delta_step' : uniform(max_delta_step_min, max_delta_step_max),
        'linear_lambda' : uniform(linear_lambda_min, linear_lambda_max),
        'min_data_per_group' : random_search_int_range(min_data_per_group_min, min_data_per_group_max),
        'max_cat_threshold' : random_search_int_range(max_cat_threshold_min, max_cat_threshold_max),
        'cat_l2' : uniform(cat_l2_min, cat_l2_max),
        'cat_smooth' : uniform(cat_smooth_min, cat_smooth_max),
        'max_cat_to_onehot' : random_search_int_range(max_cat_to_onehot_min, max_cat_to_onehot_max),
        'top_k' : random_search_int_range(top_k_min, top_k_max),
        'n_jobs' : [-1]
    }




    kfold = KFold(n_splits = 3, shuffle = True)

    random_search_accuracy = RandomizedSearchCV(lgb.LGBMClassifier(), param_distribution, n_iter = iterations, n_jobs = 10, cv=kfold, verbose=False, scoring='f1_weighted')
    random_search_accuracy.fit(x_fruit, y_fruit)

    return random_search_accuracy.best_score_

In [16]:
from scipy.stats import uniform
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV

def random_search_int_range(min_l, max_l):
    return [int(i) for i in np.arange(randint.ppf(0.01, min_l, max_l + 2),randint.ppf(0.99, min_l, max_l + 2))]


def run_random_search_auc(iterations = 1024):
    param_distribution = {
        'num_leaves' : random_search_int_range(num_leaves_min, num_leaves_max),
        'min_child_samples' : random_search_int_range(min_child_samples_min, min_child_samples_max),
        'n_estimators' : random_search_int_range(n_estimators_min, n_estimators_max),
        'learning_rate' : uniform(learning_rate_min, learning_rate_max),
        'subsample_for_bin' : random_search_int_range(subsample_for_bin_min, subsample_for_bin_max),
        'min_split_gain' : uniform(min_split_gain_min, min_split_gain_max),
        'min_child_weight' : uniform(min_child_weight_min, min_child_weight_max),
        'reg_alpha' : uniform(reg_alpha_min, reg_alpha_max),
        'reg_lambda' : uniform(reg_lambda_min, reg_lambda_max),
        'max_depth' : random_search_int_range(max_depth_min, max_depth_max),
        'bagging_fraction' : uniform(bagging_fraction_min, bagging_fraction_max),
        'pos_bagging_fraction' : uniform(pos_bagging_fraction_min, pos_bagging_fraction_max),
        'neg_bagging_fraction' : uniform(neg_bagging_fraction_min, neg_bagging_fraction_max),
        'bagging_freq' : random_search_int_range(bagging_freq_min, bagging_freq_max),
        'feature_fraction' : uniform(feature_fraction_min, feature_fraction_max),
        'feature_fraction_bynode' : uniform(feature_fraction_bynode_min, feature_fraction_bynode_max),
        'extra_trees' : [True, False],
        'max_delta_step' : uniform(max_delta_step_min, max_delta_step_max),
        'linear_lambda' : uniform(linear_lambda_min, linear_lambda_max),
        'min_data_per_group' : random_search_int_range(min_data_per_group_min, min_data_per_group_max),
        'max_cat_threshold' : random_search_int_range(max_cat_threshold_min, max_cat_threshold_max),
        'cat_l2' : uniform(cat_l2_min, cat_l2_max),
        'cat_smooth' : uniform(cat_smooth_min, cat_smooth_max),
        'max_cat_to_onehot' : random_search_int_range(max_cat_to_onehot_min, max_cat_to_onehot_max),
        'top_k' : random_search_int_range(top_k_min, top_k_max),
        'n_jobs' : [-1]
    }




    kfold = KFold(n_splits = 3, shuffle = True)

    random_search_accuracy = RandomizedSearchCV(lgb.LGBMClassifier(), param_distribution, n_iter = iterations, n_jobs = 10, cv=kfold, verbose=False, scoring='roc_auc_ovr_weighted')
    random_search_accuracy.fit(x_fruit, y_fruit)

    return random_search_accuracy.best_score_

In [17]:
from tqdm import tqdm
import pandas as pd
from os import system

def full_run_random_searh(iterations):
    filename = './lightGBM_fruit_data.csv'

    try:
        Catboost_data = pd.read_csv(filename)
    except:
        open(filename, "a")
        Catboost_data = pd.DataFrame(columns=['Algorithm', 'Accuracy', 'F1', 'AUC'])


    for i in tqdm(range(iterations)):

        accuracy_rs = run_random_search_accuracy()
        f1_rs = run_random_search_f1()
        auc_rs = run_random_search_auc()
        temp = pd.DataFrame({'Algorithm' : ['Random Search'], 
                            'Accuracy' :[accuracy_rs], 
                            'F1' : [f1_rs], 
                            'AUC' : [auc_rs]})
        #Catboost_data = Catboost_data.append(temp, ignore_index = True)
        Catboost_data = pd.concat([Catboost_data, temp], ignore_index=True)
        Catboost_data[['Algorithm', 'Accuracy', 'F1', 'AUC']].to_csv(filename)

# Gerando Dados Para Análise 

In [18]:
from tqdm import tqdm
import pandas as pd
from os import system

def full_run(iterations):
    filename = './lightGBM_fruit_data.csv'

    try:
        lightGBM_data = pd.read_csv(filename)
    except:
        open(filename, "a")
        lightGBM_data = pd.DataFrame(columns=['Algorithm', 'Accuracy', 'F1', 'AUC'])


    for i in tqdm(range(iterations)):
        #accuracy_pso, _, _ = run_accuracy_pso()
        #f1_pso, _, _ = run_f1_pso()
        #auc_pso, _, _ = run_auc_pso()
        #temp = pd.DataFrame({'Algorithm' : ['PSO'], 
        #                    'Accuracy' :[-accuracy_pso[0]], 
        #                    'F1' : [-f1_pso[0]], 
        #                    'AUC' : [-auc_pso[0]]})
        ##lightGBM_data = lightGBM_data.append(temp, ignore_index = True)
        #lightGBM_data = pd.concat([lightGBM_data, temp], ignore_index=True)
        #lightGBM_data[['Algorithm', 'Accuracy', 'F1', 'AUC']].to_csv(filename)
    
        accuracy_ga, _, _ = run_accuracy_ga()
        f1_ga, _, _ = run_f1_ga()
        auc_ga, _, _ = run_auc_ga()
        temp = pd.DataFrame({'Algorithm' : ['GA'], 
                            'Accuracy' :[-accuracy_ga[0]], 
                            'F1' : [-f1_ga[0]], 
                            'AUC' : [-auc_ga[0]]})
        #lightGBM_data = lightGBM_data.append(temp, ignore_index = True)
        lightGBM_data = pd.concat([lightGBM_data, temp], ignore_index=True)
        lightGBM_data[['Algorithm', 'Accuracy', 'F1', 'AUC']].to_csv(filename)
        
        #accuracy_optuna, _ = run_optuna_accuracy()
        #f1_optuna, _ = run_optuna_f1()
        #auc_optuna, _ = run_optuna_auc()
        #temp = pd.DataFrame({'Algorithm' : ['Optuna'], 
        #                    'Accuracy' :[accuracy_optuna], 
        #                    'F1' : [f1_optuna], 
        #                    'AUC' : [auc_optuna]})
        ##lightGBM_data = lightGBM_data.append(temp, ignore_index = True)
        #lightGBM_data = pd.concat([lightGBM_data, temp], ignore_index=True)
        #lightGBM_data[['Algorithm', 'Accuracy', 'F1', 'AUC']].to_csv(filename)

In [21]:
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)
full_run_random_searh(1)

  0%|          | 0/1 [00:00<?, ?it/s]C:\Users\Felps\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
15 fits failed out of a total of 3072.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Felps\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Felps\AppData\Local\Packages\PythonSoftwareFoundation.Python.3